## Homework 2  - Questions 5 and 6: model comparisons
Michelle Hu  
---
University of Washington  
AMATH 563  
Due: May 2, 2020  

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from scipy.special import rel_entr

In [3]:
wd = "/mnt/Backups/jmhu"

data_dir = wd + "/git_dirs/ComplexSystems_AMATH563/hw2/data/"
model_dir = wd + "/git_dirs/ComplexSystems_AMATH563/hw2/models/"
fig_dir = wd + "/git_dirs/ComplexSystems_AMATH563/hw2/figures/"
    
if not os.path.exists(data_dir):
    !mkdir $data_dir

if not os.path.exists(model_dir):
    !mkdir $model_dir

if not os.path.exists(fig_dir):
    !mkdir $fig_dir

### Load data

In [4]:
df = pd.read_csv(data_dir+"population_data.csv")
df.head()

,Year,Hare,Lynx
0,1845,20,32
1,1847,20,50
2,1849,52,12
3,1851,83,10
4,1853,64,13


### Load regression models 

In [5]:
hare_model_fns=['/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso8_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lars_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso8_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lars_hare.pkl']

lynx_model_fns=['/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso8_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lars_restricted_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lasso8_hare.pkl',
 '/mnt/Backups/jmhu/git_dirs/ComplexSystems_AMATH563/hw2/models/lars_hare.pkl']

In [6]:
hare_models=[pickle.load(open(hare_fn, 'rb')) for hare_fn in hare_model_fns]
lynx_models=[pickle.load(open(lynx_fn, 'rb')) for lynx_fn in lynx_model_fns]

In [7]:
hare_models

[Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000000,
       normalize=False, positive=False, precompute=False, random_state=43,
       selection='cyclic', tol=0.0001, warm_start=False),
 Lasso(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=100000000,
       normalize=False, positive=False, precompute=False, random_state=43,
       selection='cyclic', tol=0.0001, warm_start=False),
 LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
           fit_path=True, max_iter=500, normalize=True, positive=False,
           precompute='auto', verbose=False),
 Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000.0,
       normalize=False, positive=False, precompute=False, random_state=0,
       selection='cyclic', tol=0.0001, warm_start=False),
 Lasso(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=100000.0,
       normalize=False, positive=False, precompute=False, random_state=0,
       selection='cyclic', tol=0.0001, warm_st

In [8]:
lynx_models

[Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000000,
       normalize=False, positive=False, precompute=False, random_state=43,
       selection='cyclic', tol=0.0001, warm_start=False),
 Lasso(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=100000000,
       normalize=False, positive=False, precompute=False, random_state=43,
       selection='cyclic', tol=0.0001, warm_start=False),
 LassoLars(alpha=1.0, copy_X=True, eps=2.220446049250313e-16, fit_intercept=True,
           fit_path=True, max_iter=500, normalize=True, positive=False,
           precompute='auto', verbose=False),
 Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=100000.0,
       normalize=False, positive=False, precompute=False, random_state=0,
       selection='cyclic', tol=0.0001, warm_start=False),
 Lasso(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=100000.0,
       normalize=False, positive=False, precompute=False, random_state=0,
       selection='cyclic', tol=0.0001, warm_st

### Load DMD models
Not yet produced and pickled, once complete, merge with above

## *Best fit model is the Lasso model of the second iteration (with sinusoidal functions), based on available observations.*

---
### Question 5.  Compute the KL divergence of the best model fit to the data between all the above models.
KL divergence = statistical metric representing info lost when g(x) is used to represent f(x)

KL(f,g) = integral [ f(X,Beta) * log (f(X, Beta)/g(X, mu)) dX ]

Generate pdfs based on varying IC and model output, variance

[wiki](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)  
[scipy implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html)  
see [mlm notes](https://machinelearningmastery.com/divergence-between-probability-distributions/) for discussion on kl_div() and rel_entr()  

In [10]:
# from https://gist.github.com/swayson/86c296aa354a555536e6765bbe726ff7

def kl(p, q):
    """Kullback-Leibler divergence D(P || Q) for discrete distributions
    Parameters
    ----------
    p, q : array-like, dtype=float, shape=n
    Discrete probability distributions.
    """
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

p = [0.1, 0.9]
q = [0.1, 0.9]

kl(p, q)

0.0

Generate probability distribution functions

### Question 6. Retain three of your best fit models and compare their AIC and BIC scores.

In [11]:
def compute_AICBIC(preds, truth, k, n):
    '''
    preds = y_hat = model.predict(X)
    truth = y
    k = number of terms in model
    n = number of obserations (how big is X?)
    from class notes
    '''
    resid = truth - preds
    sse = sum(resid**2)
    aic = 2*k - 2 * np.log(sse)
    bic = np.log(n) * k - 2 * np.log(sse)
    return (aic, bic)

# Compare with
# statsmodels.tools.eval_measures.aic
# statsmodels.tools.eval_measures.bic

# and from https://pypi.org/project/RegscorePy/:
# AIC = n*log(residual sum of squares/n) + 2K
# BIC = n*log(residual sum of squares/n) + K*log(n)